In [ ]:
from google.cloud import bigquery, secretmanager

import sys
import numpy as np
import pandas as pd
import importlib
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

sys.path.append('../lib')
import utils
import pandas_utils
import pesys_utils
importlib.reload(utils)
importlib.reload(pesys_utils)
importlib.reload(pandas_utils)

In [ ]:
import dataverk_tools.themes.navdesign as theme
import dataverk_tools.themes.navdesign.plotly_template as template
pio.templates.default = template

In [ ]:
utils.set_secrets_as_env()

In [ ]:
con = pesys_utils.open_pen_connection()
df_first = pandas_utils.pandas_from_sql('../sql/auto_krav_aar.sql', con)
con.close()

In [ ]:
df_first.head()

In [ ]:
df = df_first.copy()

In [ ]:
def delauto_to_auto(x):
    if x=="DEL_AUTO": 
        return "AUTO"
    else: return x

In [ ]:
df = df[(df.SAKSTYPE == 'Alderspensjon') & (df.KRAVTYPE.isin(pesys_utils.forstegang))]

df["AUTOMATISERING"] = df.AUTOMATISERING.apply(delauto_to_auto)

In [ ]:
def group_df_on_var(df, period, var):
    if type(var) is list or type(var) is tuple:
        df = df.groupby([period] + var)[period].count().reset_index(name="ANTALL")
    else:
        df = df.groupby([period, var])[var].count().reset_index(name="ANTALL")

    df = df.merge(df.groupby([period], as_index=False).ANTALL.sum(),
                    left_on=[period],
                    right_on=[period],
                    suffixes=(""," TOTALT")
            )
    df["ANDEL"] = df.ANTALL.divide(df["ANTALL TOTALT"])
    return df

In [ ]:
df_aar = df.groupby(["AUTOMATISERING", "AAR"], as_index=False).ANTALL.sum()
df_aar = df_aar[(df_aar.AAR >= 2011) & (df_aar.AAR < 2023)] #Dårlig data før 2011
df_aar = df_aar.merge(df_aar.groupby(["AAR"], as_index=False).ANTALL.sum(),
                    left_on=["AAR"],
                    right_on=["AAR"],
                    suffixes=(""," TOTALT")
            )
df_aar["ANDEL"] = df_aar.ANTALL.divide(df_aar["ANTALL TOTALT"])

In [ ]:
df_aar.head()

In [ ]:
df_aar.sort_values(["AAR", "AUTOMATISERING"], inplace=True)

In [ ]:
px.bar(df_aar, "AAR", "ANTALL", color="AUTOMATISERING")
#fig.add_traces(go.Scatter(x=df_aar[df_aar["AUTOMATISERING"] == "AUTO"].AAR, y=df_aar[df_aar["AUTOMATISERING"] == "AUTO"].ANDEL))

In [ ]:
px.line(df_aar, "AAR", "ANDEL", color="AUTOMATISERING")

In [ ]:
vekst_fra_2021 = np.array([1.00, 1.02, 1.04, 1.06, 1.08, 1.12, 1.15, 1.18, 1.21, 1.24, 1.27, 1.29, 1.32, 1.35, 1.38, 1.40, 1.43, 1.45, 1.47, 1.49, 1.51, 1.53, 1.55, 1.56, 1.58, 1.59, 1.61, 1.62, 1.64, 1.65])

In [ ]:
år = np.array(range(2021,2051,1))

In [ ]:
df_2021 = df_aar[df_aar.AAR == 2021]

In [ ]:
df_2021[(df_2021.AUTOMATISERING == 'AUTO') | (df_2021.AUTOMATISERING == 'DEL_AUTO')].ANTALL.sum() / df_2021.ANTALL.sum()

In [ ]:
df_2021.ANTALL.sum()

In [ ]:
df_forecast = pd.DataFrame(data = {"År": år, "Vekst": vekst_fra_2021} )

In [ ]:
df_forecast["Manuelt"] = df_forecast.Vekst * df_2021[df_2021.AUTOMATISERING == 'MAN'].ANTALL.item()
df_forecast["Automatisk"] = df_forecast.Vekst * df_2021[df_2021.AUTOMATISERING == 'AUTO'].ANTALL.item()
#df_forecast["Del-automatisk"] = df_forecast.Vekst * df_2021[df_2021.AUTOMATISERING == 'DEL_AUTO'].ANTALL.item()

In [ ]:
df_forecast.head()

In [ ]:
p0 = df_2021[(df_2021.AUTOMATISERING == 'AUTO') | (df_2021.AUTOMATISERING == 'DEL_AUTO')].ANTALL.sum() / df_2021.ANTALL.sum().item()
df_forecast["Automatiseringsgrad"] = - (1 - p0 - df_forecast.Vekst) / df_forecast.Vekst
df_forecast["Forklaring1"] = "Utvikling for samme mengde manuelt arbeid som 2021"
df_forecast["Forklaring2"] = "Med samme automatiseringsgrad som i dag"

In [ ]:
df_forecast.head()

In [ ]:
df_aar_auto = df_aar[(df_aar.AUTOMATISERING == 'AUTO') & (df_aar.AAR > 2017)]

fig = px.line(df_forecast, "År", "Automatiseringsgrad", line_dash="Forklaring1", line_dash_sequence=['dash'], 
              title="Andel automatisk eller del-automatisk behandling - førstegangsbehandling alderspensjon")

fig.add_traces(go.Scatter(x=df_aar_auto.AAR, y=df_aar_auto.ANDEL, name="Utvikling siden 2017"))

fig.update_yaxes(tickformat='.0%', range=[0.6,.85])
fig.update_layout(legend_title="")

In [ ]:
df_aar_man = df_aar[(df_aar.AUTOMATISERING == 'MAN') & (df_aar.AAR > 2016)]

fig_man = px.bar(df_forecast[df_forecast.År > 2022], "År", "Manuelt", color="Forklaring2", 
                 labels={"Manuelt":"Antall manuelle krav"}, title="Antall manuelt behandlede krav - førstegangsbehandling alderpensjon")

fig_man.add_traces(go.Bar(x=df_aar_man.AAR, y=df_aar_man.ANTALL, name="Siden 2017"))

fig_man.update_layout(legend_title="")
#fig_man.add_hline(y=21000, line_dash="dash", line_color="lightblue")

In [ ]:
n_man_2021 = df_2021[df_2021.AUTOMATISERING == 'MAN'].ANTALL.item()

In [ ]:
df_forecast["diff_siden_2021"] = df_forecast["Manuelt"] - n_man_2021

In [ ]:
df_forecast.diff_siden_2021.sum()

In [ ]:
df_forecast[df_forecast.År <= 2030].diff_siden_2021.sum()